In [10]:
import pandas as pd
import numpy as np

# Load CSVs
job_postings = pd.read_csv("../data/job_postings.csv")
companies = pd.read_csv("../data/companies.csv")
job_skills = pd.read_csv("../data/job_skills.csv")
benefits = pd.read_csv("../data/benefits.csv")
company_industries = pd.read_csv("../data/company_industries.csv")
company_specialities = pd.read_csv("../data/company_specialities.csv")
employee_counts = pd.read_csv("../data/employee_counts.csv")
job_industries = pd.read_csv("../data/job_industries.csv")



In [11]:
import pandas as pd

# List of your DataFrames
dataframes = {
    'job_postings': job_postings,
    'companies': companies,
    'job_skills': job_skills,
    'benefits': benefits,
    'company_industries': company_industries,
    'company_specialities': company_specialities,
    'employee_counts': employee_counts,
    'job_industries': job_industries
}

# Loop through all DataFrames to check their types and basic info

print("--- Data Type and Info Check for All Loaded DataFrames ---")
print("-" * 50)

for name, df in dataframes.items():
    print(f"\n✅ DataFrame: {name}")
    print("-----------------------------------")

    # Option 1: Quick DTypes printout
    print(df.dtypes)
    
    # Option 2: Detailed Info (recommended for checking nulls and types together)
    # df.info() 
    # Uncomment the line above if you want the full memory usage and non-null count summary.
    
    print("-" * 50)

--- Data Type and Info Check for All Loaded DataFrames ---
--------------------------------------------------

✅ DataFrame: job_postings
-----------------------------------
job_id                          int64
company_id                    float64
title                          object
description                    object
max_salary                    float64
med_salary                    float64
min_salary                    float64
pay_period                     object
formatted_work_type            object
location                       object
applies                       float64
original_listed_time          float64
remote_allowed                float64
views                         float64
job_posting_url                object
application_url                object
application_type               object
expiry                        float64
closed_time                   float64
formatted_experience_level     object
skills_desc                    object
listed_time                  

In [12]:
import pandas as pd

# =================================================================
# 1. job_postings DataFrame Conversions
# =================================================================

print("--- Converting job_postings dtypes ---")

# CRITICAL FIX: Convert company_id from float64 to nullable integer (Int64) 
# to match the int64 type in the 'companies' table for merging.
# We use 'Int64' (capital I) to handle potential NaN values.
job_postings['company_id'] = job_postings['company_id'].astype('Int64')

# Convert time-related floats to datetime objects for analysis.
# Assuming timestamps are in milliseconds ('ms').
date_cols = ['original_listed_time', 'listed_time', 'closed_time', 'expiry']
for col in date_cols:
    # Create new datetime columns
    job_postings[col.replace('_time', '_date')] = pd.to_datetime(
        job_postings[col], unit='ms', errors='coerce'
    )
    # Drop the original float columns to avoid confusion
    job_postings.drop(columns=[col], inplace=True)
    
# Convert count-related floats to nullable integers (Int64).
count_cols = ['applies', 'views', 'remote_allowed']
for col in count_cols:
    job_postings[col] = job_postings[col].astype('Int64')

print(job_postings.dtypes)


# =================================================================
# 2. companies DataFrame Conversions
# =================================================================

print("\n--- Converting companies dtypes ---")

# company_size is a float. If it represents a numeric code or size category, 
# converting it to nullable Int64 helps, though object (string) might be better 
# if the original text was lost. We'll stick to Int64 for numerical consistency.
companies['company_size'] = companies['company_size'].astype('Int64')

# job_id and name are already correct (int64 and object).
print(companies.dtypes)


# =================================================================
# 3. employee_counts DataFrame Conversions
# =================================================================

print("\n--- Converting employee_counts dtypes ---")

# employee_count and follower_count are already int64 (good).
# Convert time_recorded float to datetime object.
employee_counts['time_recorded_date'] = pd.to_datetime(
    employee_counts['time_recorded'], unit='ms', errors='coerce'
)
employee_counts.drop(columns=['time_recorded'], inplace=True)

print(employee_counts.dtypes)


# =================================================================
# 4. DataFrames with No Critical Conversions Needed
# =================================================================

# The following DataFrames have clean key columns (int64) and require no
# immediate datatype changes before merging:
# - job_skills (job_id: int64, skill_abr: object)
# - benefits (job_id: int64, inferred: int64, type: object)
# - company_industries (company_id: int64, industry: object)
# - company_specialities (company_id: int64, speciality: object)
# - job_industries (job_id: int64, industry_id: int64)

print("\nAll necessary pre-merge datatype conversions are complete.")
print("You can now safely proceed to the merging step!")

--- Converting job_postings dtypes ---
job_id                                 int64
company_id                             Int64
title                                 object
description                           object
max_salary                           float64
med_salary                           float64
min_salary                           float64
pay_period                            object
formatted_work_type                   object
location                              object
applies                                Int64
remote_allowed                         Int64
views                                  Int64
job_posting_url                       object
application_url                       object
application_type                      object
formatted_experience_level            object
skills_desc                           object
posting_domain                        object
sponsored                              int64
work_type                             object
currency        

In [13]:
import pandas as pd

# --- CRITICAL PRE-MERGE CLEANING FOR employee_counts ---
# The employee_counts table has multiple time-stamped entries per company_id.
# We must ensure only ONE, the LATEST, record is used for merging to avoid duplication.

# 1. Sort by company_id and time_recorded_date (descending)
employee_counts_sorted = employee_counts.sort_values(
    ['company_id', 'time_recorded_date'],
    ascending=[True, False] # Sort by Company ID (Asc), then by Date (Descending - newest first)
)

# 2. Drop duplicates, keeping the first (most recent) record for each company
employee_counts_unique = employee_counts_sorted.drop_duplicates(
    subset=['company_id'],
    keep='first'
)

print(f"Employee Counts reduced from {len(employee_counts)} to {len(employee_counts_unique)} unique company records.")

# --- START OF MASTER MERGE PROCESS ---

# 1. Merge Job Postings (Left side) with Company Details (companies)
master_df = pd.merge(
    job_postings,
    companies[['company_id', 'name', 'company_size', 'city', 'country']],
    on='company_id',
    how='left',
    suffixes=('_job', '_company')
)

print("\nMerged job_postings with companies.")
print(f"Current master_df shape: {master_df.shape}")

# 2. Prepare Skills Data (Many-to-One transformation)
# Group all skills for a single job_id into a single string.
# We explicitly cast to string (astype(str)) just in case 'skill_abr' had non-string types.
skills_grouped = job_skills.groupby('job_id')['skill_abr'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
skills_grouped.rename(columns={'skill_abr': 'all_skills_list'}, inplace=True)

# 3. Merge Skills Data into the Master Table
master_df = pd.merge(
    master_df,
    skills_grouped,
    on='job_id',
    how='left'
)

print("Merged job_skills into master_df.")
print(f"Current master_df shape: {master_df.shape}") # Should still be (15886, 31) or similar

# 4. Merge Employee Counts Data (CORRECTED STEP)
# Use the cleaned, unique employee counts table here!
master_df = pd.merge(
    master_df,
    employee_counts_unique[['company_id', 'employee_count', 'follower_count', 'time_recorded_date']],
    on='company_id',
    how='left'
)

print("Merged employee_counts into master_df.")
print(f"Final master_df shape: {master_df.shape}")

# --- FINAL VERIFICATION ---
# Expected shape is close to the starting job posting count (15886)
# If the shape is 15886, the merge was successful.

print("\n--- Master DataFrame Head ---")
print(master_df.head())

Employee Counts reduced from 15907 to 6030 unique company records.

Merged job_postings with companies.
Current master_df shape: (15886, 30)
Merged job_skills into master_df.
Current master_df shape: (15886, 31)
Merged employee_counts into master_df.
Final master_df shape: (15886, 34)

--- Master DataFrame Head ---
      job_id  company_id                     title  \
0   85008768        <NA>  Licensed Insurance Agent   
1  133114754    77766802             Sales Manager   
2  133196985     1089558        Model Risk Auditor   
3  381055942    96654609          Business Manager   
4  529257371     1244539       NY Studio Assistant   

                                         description  max_salary  med_salary  \
0  While many industries were hurt by the last fe...     52000.0         NaN   
1  Are you a dynamic and creative marketing profe...         NaN         NaN   
2  Join Us as a Model Risk Auditor – Showcase You...         NaN         NaN   
3  Business ManagerFirst Baptist Churc

In [14]:
# --- 1. Job Title Normalization ---

# Convert to lowercase and remove common noise
master_df['title_clean'] = master_df['title'].str.lower()
master_df.dropna(subset=['title_clean'], inplace=True) # Drop if title is missing

# A. Standardize Seniority and Levels (using regex)
# \b ensures we match the whole word/abbreviation
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\b(sr|senior)\b\.?', 'senior', regex=True)
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\b(jr|junior)\b\.?', 'junior', regex=True)
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\b(lead|manager|director)\b', '', regex=True).str.strip()


# B. Group Core Data Science/Analytics Roles
# Standardize common titles into one core term (e.g., all things 'engineer' become 'scientist')
replacements = {
    r'machine learning': 'ml',
    r'data science engineer': 'data scientist',
    r'data engineer': 'data engineering',
    r'business intelligence': 'bi analyst',
    r'bi developer': 'bi analyst',
    r'data analyst': 'data analyst',
    r'data science': 'data scientist'
}

for pattern, replacement in replacements.items():
    master_df['title_clean'] = master_df['title_clean'].str.replace(pattern, replacement, regex=False).str.strip()
    
# Clean up any extra whitespace from replacements
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\s+', ' ', regex=True)

print(f"Master DataFrame shape after title cleaning: {master_df.shape}")
print("\nTop 10 Cleaned Job Titles:")
print(master_df['title_clean'].value_counts().head(10))

Master DataFrame shape after title cleaning: (15886, 35)

Top 10 Cleaned Job Titles:
title_clean
sales [owner/operator]             83
sales {owner/operator}             78
sales associate                    69
retail sales associate             60
sales                              44
project                            42
staff accountant                   42
administrative assistant           40
senior accountant                  40
customer service representative    39
Name: count, dtype: int64


In [15]:
# --- 2. Salary Feature Engineering ---

# A. Filter for only YEARLY pay periods
# This creates the subset you will use for your prediction model.
salary_df = master_df[master_df['pay_period'] == 'YEARLY'].copy()

# B. Calculate the midpoint for the annual salary
# Check if both min and max are not null before calculating.
salary_df['annual_salary'] = (salary_df['min_salary'] + salary_df['max_salary']) / 2

# C. Handle Missing Values
# Drop rows where we still couldn't determine an annual salary
salary_df.dropna(subset=['annual_salary'], inplace=True)

# D. Basic Outlier Removal (Recommended for salary)
# Filter for reasonable bounds (e.g., only keep salaries between the 1st and 99th percentile)
Q1 = salary_df['annual_salary'].quantile(0.01)
Q3 = salary_df['annual_salary'].quantile(0.99)
salary_df = salary_df[(salary_df['annual_salary'] >= Q1) & (salary_df['annual_salary'] <= Q3)]


# E. Drop redundant salary columns from the prediction subset
salary_df.drop(
    columns=['max_salary', 'min_salary', 'med_salary', 'pay_period', 'currency', 'compensation_type'],
    inplace=True
)

print(f"\nDataFrame shape for SALARY PREDICTION model: {salary_df.shape}")
print(f"Median Annual Salary: ${salary_df['annual_salary'].median():,.2f}")


DataFrame shape for SALARY PREDICTION model: (3575, 30)
Median Annual Salary: $101,050.00


In [16]:
# --- 3. Final Preparation and Saving ---

# A. Handle other missing values
# For text features that will be used in NLP, fill NaNs with an empty string
master_df['all_skills_list'].fillna('', inplace=True)
master_df['description'].fillna('', inplace=True)

# B. Save the fully cleaned master data
# Use this for all analysis (EDA) and the Clustering/NLP models.
master_df.to_csv('cleaned_master_data.csv', index=False)

# C. Save the salary-specific data
# Use this ONLY for the Regression Prediction model.
salary_df.to_csv('cleaned_salary_data.csv', index=False)

print("\nSaved two clean datasets: 'cleaned_master_data.csv' (for EDA/NLP) and 'cleaned_salary_data.csv' (for Regression).")

# You are now ready for Week 2: Data Cleaning & EDA!

C:\Users\lenovo\AppData\Local\Temp\ipykernel_840\4068405783.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df['all_skills_list'].fillna('', inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_840\4068405783.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 


Saved two clean datasets: 'cleaned_master_data.csv' (for EDA/NLP) and 'cleaned_salary_data.csv' (for Regression).


In [17]:
# Assuming master_df is your current DataFrame

# Fix the FutureWarning by assigning the result back, instead of using inplace=True
master_df['all_skills_list'] = master_df['all_skills_list'].fillna('')
master_df['description'] = master_df['description'].fillna('')

print("FutureWarnings resolved. DataFrames are ready.")

FutureWarnings resolved. DataFrames are ready.


In [18]:
# Assuming master_df is your current DataFrame

# Convert to lowercase and remove common noise
master_df['title_clean'] = master_df['title'].str.lower()
master_df.dropna(subset=['title_clean'], inplace=True) 

# A. Standardize Seniority and Levels (using regex)
# This keeps the titles clean without changing their category
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\b(sr|senior)\b\.?', 'senior', regex=True)
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\b(jr|junior)\b\.?', 'junior', regex=True)
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\b(lead|manager|director)\b', '', regex=True).str.strip()
master_df['title_clean'] = master_df['title_clean'].str.replace(r'[\{\}\[\]\/\\]', ' ', regex=True).str.strip()

# Final cleanup of extra whitespace
master_df['title_clean'] = master_df['title_clean'].str.replace(r'\s+', ' ', regex=True)

print("\n--- Top 10 Job Titles (Simple Clean-Up) ---")
print(master_df['title_clean'].value_counts().head(10))

# Re-save the final cleaned dataframes
master_df.to_csv('cleaned_master_data.csv', index=False)
# The salary_df was already saved and doesn't need re-saving unless you re-created it.


--- Top 10 Job Titles (Simple Clean-Up) ---
title_clean
sales owner operator               161
sales associate                     69
retail sales associate              60
sales                               44
project                             42
staff accountant                    42
senior accountant                   40
administrative assistant            40
customer service representative     39
call center support rep             38
Name: count, dtype: int64
